In [1]:
from PIL import Image 
from IPython.display import display 
import random
import json

In [2]:
# Each image is made up a series of traits
# The weightings for each trait drive the rarity and add up to 100%

background = ["blue", "orange", "purple", "red", "yellow"] 
background_weights = [30, 40, 15, 5, 10]

hair = ["blue", "orange", "purple", "red", "yellow"] 
hair_weights = [30, 40, 15, 5, 10]

cover = ["casual", "cool", "fancy", "love", "rock"] 
cover_weights = [30, 40, 15, 5, 10]

# Dictionary variable for each trait. 
# Eech trait corresponds to its file name

background_files = {
    "blue": "blue",
    "orange": "orange",
    "purple": "purple",
    "red": "red",
    "yellow": "yellow",
}

hair_files = {
    "blue": "blue-hair",
    "orange": "orange-hair",
    "purple": "purple-hair",
    "red": "red-hair",
    "yellow": "yellow-hair",   
}

cover_files = {
    "casual": "casual-cover",
    "cool": "cool-cover",
    "fancy": "fancy-cover",
    "love": "love-cover",
    "rock": "rock-cover"            
}

In [3]:
## Generate Traits

TOTAL_IMAGES = 100 # Number of random unique images we want to generate

all_images = [] 

# A recursive function to generate unique image combinations
def create_new_image():
    
    new_image = {} #

    # For each trait category, select a random trait based on the weightings 
    new_image ["background"] = random.choices(background, background_weights)[0]
    new_image ["hair"] = random.choices(hair, hair_weights)[0]
    new_image ["cover"] = random.choices(cover, cover_weights)[0]
    
    if new_image in all_images:
        return create_new_image()
    else:
        return new_image
    
    
# Generate the unique combinations based on trait weightings
for i in range(TOTAL_IMAGES): 
    
    new_trait_image = create_new_image()
    
    all_images.append(new_trait_image)

In [4]:
# Returns true if all images are unique
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

Are all images unique? True


In [5]:
# Add token Id to each image
i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

In [6]:
print(all_images)

[{'background': 'purple', 'hair': 'blue', 'cover': 'cool', 'tokenId': 0}, {'background': 'blue', 'hair': 'blue', 'cover': 'cool', 'tokenId': 1}, {'background': 'blue', 'hair': 'orange', 'cover': 'cool', 'tokenId': 2}, {'background': 'orange', 'hair': 'blue', 'cover': 'fancy', 'tokenId': 3}, {'background': 'purple', 'hair': 'purple', 'cover': 'cool', 'tokenId': 4}, {'background': 'orange', 'hair': 'blue', 'cover': 'casual', 'tokenId': 5}, {'background': 'orange', 'hair': 'orange', 'cover': 'casual', 'tokenId': 6}, {'background': 'purple', 'hair': 'yellow', 'cover': 'cool', 'tokenId': 7}, {'background': 'blue', 'hair': 'orange', 'cover': 'rock', 'tokenId': 8}, {'background': 'blue', 'hair': 'yellow', 'cover': 'cool', 'tokenId': 9}, {'background': 'blue', 'hair': 'red', 'cover': 'casual', 'tokenId': 10}, {'background': 'purple', 'hair': 'orange', 'cover': 'cool', 'tokenId': 11}, {'background': 'orange', 'hair': 'orange', 'cover': 'fancy', 'tokenId': 12}, {'background': 'blue', 'hair': 'or

In [7]:
# Get Trait Counts

background_count = {}
for item in background:
    background_count[item] = 0
    
hair_count = {}
for item in hair:
    hair_count[item] = 0

cover_count = {}
for item in cover:
    cover_count[item] = 0

for image in all_images:
    background_count[image["background"]] += 1
    hair_count[image["hair"]] += 1
    cover_count[image["cover"]] += 1
    
print(background_count)
print(hair_count)
print(cover_count)

{'blue': 23, 'orange': 24, 'purple': 20, 'red': 16, 'yellow': 17}
{'blue': 23, 'orange': 23, 'purple': 21, 'red': 16, 'yellow': 17}
{'casual': 24, 'cool': 22, 'fancy': 22, 'love': 15, 'rock': 17}


In [8]:
#### Generate Metadata for all Traits 
METADATA_FILE_NAME = 'updateMetadataOpenSea.js';
with open(METADATA_FILE_NAME, 'w') as outfile:
    json.dump(all_images, outfile, indent=4)

In [9]:
#### Generate Images    
for item in all_images:

    im1 = Image.open(f'./background/{background_files[item["background"]]}.jpg').convert('RGBA')
    im2 = Image.open(f'./hair/{hair_files[item["hair"]]}.png').convert('RGBA')
    im3 = Image.open(f'./cover/{cover_files[item["cover"]]}.png').convert('RGBA')


    #Create each composite
    com1 = Image.alpha_composite(im1, im2)
    com2 = Image.alpha_composite(com1, im3)

    #Convert to RGB
    rgb_im = com2.convert('RGB')
    file_name = str(item["tokenId"]) + ".png"
    rgb_im.save("./images/" + file_name)

In [13]:
#### Generate Metadata for each Image    

f = open('updateMetadataOpenSea.js',) 
data = json.load(f)


IMAGES_BASE_URI = "https://github.com/yourboyjus/itsme.git"
PROJECT_NAME = "Girls"

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
for i in data:
    token_id = i['tokenId']
    token = {
        "image": IMAGES_BASE_URI + str(token_id) + '.png',
        "tokenId": token_id,
        "name": PROJECT_NAME + ' ' + str(token_id),
        "attributes": []
    }
    token["attributes"].append(getAttribute("background", i["background"]))
    token["attributes"].append(getAttribute("hair", i["hair"]))
    token["attributes"].append(getAttribute("cover", i["cover"]))

    with open('./metadata/' + str(token_id), 'w') as outfile:
        json.dump(token, outfile, indent=4)
f.close()